Importazione delle librerie necessarie

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

In [2]:
raw_folder_path = os.path.join("", "raw")
graphics_folder_path = os.path.join('', 'graphics')
data_folder_path = os.path.join('', 'data')

happiness_file_name = "World-happiness-report-2024.csv"
population_file_name = "world_population.csv"
gdp_file_name = "country_data.csv"
literacy_file_name = "Literacy Rate.csv"

happiness_file_path = os.path.join(raw_folder_path, str(happiness_file_name))
population_file_path = os.path.join(raw_folder_path, str(population_file_name))
gdp_file_path = os.path.join(raw_folder_path, str(gdp_file_name))
literacy_file_path = os.path.join(raw_folder_path, str(literacy_file_name))

if not os.path.exists(raw_folder_path):
    os.makedirs(raw_folder_path)

if not os.path.exists(graphics_folder_path):
    os.makedirs(graphics_folder_path)

if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

if os.path.exists(happiness_file_path) and os.path.exists(population_file_path) and os.path.exists(gdp_file_path) and os.path.exists(literacy_file_path):
    happiness = pd.read_csv(happiness_file_path)
    population = pd.read_csv(population_file_path)
    gdp = pd.read_csv(gdp_file_path)
    literacy = pd.read_csv(literacy_file_path)
else:
    raise Exception

In [3]:
population.rename(columns={"Country/Territory": "country", "2022 Population": "population"}, inplace=True)
happiness.rename(columns={"Country name": "country"}, inplace=True)
gdp.rename(columns={"name": "country"}, inplace=True)

happiness.drop(columns=["Log GDP per capita", "Healthy life expectancy"], inplace=True)
gdp.drop(columns=["population", "capital", "pop_growth", "pop_density"], inplace=True)
population.drop(columns=["2020 Population", "2015 Population", "2010 Population", "2000 Population", "1990 Population", "1980 Population", "1970 Population"], inplace=True)

literacy = literacy.sort_values('Year', ascending=False).drop_duplicates(subset='Country', keep='first')
literacy.rename(columns={"Country": "country"}, inplace=True)

In [4]:
tmp = pd.merge(happiness, gdp, on="country", how="inner")
tmp2 = pd.merge(tmp, population, on="country", how="inner")
data = pd.merge(tmp2, literacy, on="country", how="inner")

data.columns = [column.replace('_', ' ').capitalize() for column in data.columns]
data.rename(columns={"Ladder score": "Happiness", "Gdp per capita": "GDP per capita"}, inplace=True)

data.dropna(subset=["Country", "Happiness", "GDP per capita", "Life expectancy female", "Life expectancy male", "Unemployment", "Homicide rate", "Growth rate",
                    "Density (per km²)", "Population", "Literacy rate"], inplace=True)

data["Life expectancy"] = data["Life expectancy male"] + data["Life expectancy female"] / 2



In [5]:
data.to_pickle(os.path.join(data_folder_path, "data.pickle"))